## Model Selection

This notebook should include preliminary and baseline modeling.
- Try as many different models as possible.
- Don't worry about hyperparameter tuning or cross validation here.
- Ideas include:
    - linear regression
    - support vector machines
    - random forest
    - xgboost

In [1]:
# import models and fit
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.decomposition import PCA

In [2]:
DataPath = '../data/processed/SplitAndScaled.pkl'

with open(DataPath, 'rb') as f:
    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = pickle.load(f)

In [3]:
# reducing the amount of features with PCA
PCA30 = PCA(n_components=30)
X_train_reduced = PCA30.fit_transform(X_train_scaled)
X_test_reduced = PCA30.transform(X_test_scaled)

In [4]:
# number of observations
NumObs = len(y_test_scaled)

# number of predictors
k_reduced = X_train_reduced.shape[1]

In [5]:
# fitting linear regression to training data
LinReg = LinearRegression()
LinReg.fit(X_train_reduced,y_train_scaled)

# predict the test set results
LinReg_y_pred = LinReg.predict(X_test_reduced)

# check accuracy
LinReg_MSE = mean_squared_error(y_test_scaled, LinReg_y_pred)
LinReg_RMSE = np.sqrt(LinReg_MSE)
LinRegR2 = r2_score(y_test_scaled, LinReg_y_pred)
LinRegR2Adj = 1 - (1 - LinRegR2) * (NumObs - 1) / (NumObs - k_reduced - 1)

print(f'RMSE: {LinReg_RMSE}, R2: {LinRegR2}, AdjR2: {LinRegR2Adj}')

RMSE: 0.617378085562288, R2: 0.47833431184817343, AdjR2: 0.43920938523678643


In [6]:
# fitting polynomial regression to training data
PolReg = PolynomialFeatures(degree = 2) 
X_Polynomial = PolReg.fit_transform(X_train_reduced)

LinRegForP = LinearRegression()
LinRegForP.fit(X_Polynomial, y_train_scaled)

# predict the test set results
PolReg_y_pred = LinRegForP.predict(PolReg.fit_transform(X_test_reduced))

# check accuracy
PolReg_MSE = mean_squared_error(y_test_scaled, PolReg_y_pred)
PolReg_RMSE = np.sqrt(PolReg_MSE)
PolRegR2 = r2_score(y_test_scaled, PolReg_y_pred)
PolRegR2Adj = 1 - (1 - PolRegR2) * (NumObs - 1) / (NumObs - k_reduced - 1)

print(f'RMSE: {PolReg_RMSE}, R2: {PolRegR2}, AdjR2: {PolRegR2Adj}')

RMSE: 1.3088262697083732, R2: -1.3445195557316607, AdjR2: -1.5203585224115352


In [7]:
# instantiate Random Forest Regressor
RFR = RandomForestRegressor(random_state=0)

# reshape y_train_scaled to a 1D array
reshape_y_train_scaled = y_train_scaled.ravel()

# fit the model
RFR.fit(X_train_reduced, reshape_y_train_scaled)

# predict the test set results
RFR_y_pred = RFR.predict(X_test_reduced)

# check accuracy
RFR_MSE = mean_squared_error(y_test_scaled,RFR_y_pred)
RFR_RMSE = np.sqrt(RFR_MSE)
RFR_R2 = r2_score(y_test_scaled, RFR_y_pred)
RFR_R2Adj = 1 - (1 - RFR_R2) * (NumObs - 1) / (NumObs - k_reduced - 1)

print(f'RMSE: {RFR_RMSE}, R2: {RFR_R2}, AdjR2: {RFR_R2Adj}')

RMSE: 0.6008274902635067, R2: 0.5059289088152876, AdjR2: 0.46887357697643417


In [8]:
# instantiate XGBoost model
XGB_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=0)

# fit the model
XGB_model.fit(X_train_reduced, y_train_scaled)

# predict the test set results
XGB_y_pred = XGB_model.predict(X_test_reduced)

# check accuracy
XGB_MSE = mean_squared_error(y_test_scaled, XGB_y_pred)
XGB_RMSE = np.sqrt(XGB_MSE)
XGB_R2 = r2_score(y_test_scaled, XGB_y_pred)
XGB_R2Adj = 1 - (1 - XGB_R2) * (NumObs - 1) / (NumObs - k_reduced - 1)

print(f'RMSE: {XGB_RMSE}, R2: {XGB_R2}, AdjR2: {XGB_R2Adj}')

RMSE: 0.6101807113685882, R2: 0.49042653737384456, AdjR2: 0.4522085276768829


In [10]:
# saving models
MainPath = '../data/model/'

# Linear Regression
LinRegPath = 'LinReg_model.sav'
with open(MainPath + LinRegPath, 'wb') as f:
    pickle.dump((LinReg, X_train_reduced, X_test_reduced, y_train_scaled, y_test_scaled), f)

# Polynomial Regression
PolRegPath = 'PolReg_model.sav'
with open(MainPath + PolRegPath, 'wb') as f:
    pickle.dump((LinRegForP, X_train_reduced, X_test_reduced, y_train_scaled, y_test_scaled), f)

# Random Forest Regresson
RFRPath = 'RFR_model.sav'
with open(MainPath + RFRPath, 'wb') as f:
    pickle.dump((RFR, X_train_reduced, X_test_reduced, y_train_scaled, y_test_scaled), f)

# XGBoost
XGBPath = 'XGB_model.sav'
with open(MainPath + XGBPath, 'wb') as f:
    pickle.dump((XGB_model, X_train_reduced, X_test_reduced, y_train_scaled, y_test_scaled), f)

Consider what metrics you want to use to evaluate success.
- If you think about mean squared error, can we actually relate to the amount of error?
- Try root mean squared error so that error is closer to the original units (dollars)
- What does RMSE do to outliers?
- Is mean absolute error a good metric for this problem?
- What about R^2? Adjusted R^2?
- Briefly describe your reasons for picking the metrics you use

In [11]:
# gather evaluation metrics and compare results
print(f'LinReg_RMSE: {LinReg_RMSE}, LinRegR2: {LinRegR2}, LinRegR2Adj: {LinRegR2Adj}')
print(f'PolReg_RMSE: {PolReg_RMSE}, PolRegR2: {PolRegR2}, PolRegR2Adj: {PolRegR2Adj}')
print(f'RFR_RMSE: {RFR_RMSE}, RFR_R2: {RFR_R2}, RFR_R2Adj: {RFR_R2Adj}')
print(f'XGB_RMSE: {XGB_RMSE}, XGB_R2: {XGB_R2}, XGB_R2Adj: {XGB_R2Adj}')

LinReg_RMSE: 0.617378085562288, LinRegR2: 0.47833431184817343, LinRegR2Adj: 0.43920938523678643
PolReg_RMSE: 1.3088262697083732, PolRegR2: -1.3445195557316607, PolRegR2Adj: -1.5203585224115352
RFR_RMSE: 0.6008274902635067, RFR_R2: 0.5059289088152876, RFR_R2Adj: 0.46887357697643417
XGB_RMSE: 0.6101807113685882, XGB_R2: 0.49042653737384456, XGB_R2Adj: 0.4522085276768829


<div style="color: pink;">
<h2>Interpretation:</h2>
<p>I used RMSE and R2. </p>

<p>Mean Squared Error depicts more error than there actually is due to squaring the errors.Mean Absolute Error weighs all errors equally which can be an issue since we have a range of different selling prices.</p>

<p>Root Mean Squared Error doesn't have the issue that MSE does. It also is able to point out large errors. Since the data is scaled from 0 to 1, a RMSE over 1 is incredibly high. R2 compares the model against a constant. It doesn't account for the number of features Adjusted R2 adjusts for the amount of features. </p>

<p>The Polynomial Regression model doesn't appear to be able to explain any of the data, based on the negative R2 and adjusted R2.</p>

<p>Based on the Adjusted R2, the model that explains the most of the data is the Random Forest Regression. It has an Adjusted R2 of 0.469, explaining 46.9% of the data.  Random Forest Regression also has the lowest Root Mean Square Error, which indicates less error than the other models.</p>

</div>

## Feature Selection - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its recommended you complete it if you have time!**

Even with all the preprocessing we did in Notebook 1, you probably still have a lot of features. Are they all important for prediction?

Investigate some feature selection algorithms (Lasso, RFE, Forward/Backward Selection)
- Perform feature selection to get a reduced subset of your original features
- Refit your models with this reduced dimensionality - how does performance change on your chosen metrics?
- Based on this, should you include feature selection in your final pipeline? Explain

Remember, feature selection often doesn't directly improve performance, but if performance remains the same, a simpler model is often preferrable. 



In [ ]:
# perform feature selection 
# refit models
# gather evaluation metrics and compare to the previous step (full feature set)